In [1]:
import pyspark
from pyspark.ml.feature import MinMaxScaler, StandardScaler, VectorAssembler
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql import functions as F
from pyspark.sql import types as T
import yaml
import os


In [3]:
SEED = 42

config = yaml.safe_load(open("../config.yaml", "r"))

spark = (
    SparkSession.builder.appName("Medallion Architecture")
    .config("spark.executor.memory", config["total_memory"])
    .config("spark.driver.memory", config["total_memory"])
    .config("spark.executor.instances", "1")
    .config("spark.executor.cores", str(config["total_cores"] - 1))
    .config("spark.sql.shuffle.partitions", str(config["total_cores"] * 2))
    .config("spark.memory.fraction", "0.8")
    .config("spark.memory.storageFraction", "0.3")
    .getOrCreate()
)


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/25 18:08:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
def bronze_layer(
    raw_input_path: str, bronze_output_path: str, bronze_sql_name: str
) -> None:
    df = spark.read.csv(raw_input_path, header=True, inferSchema=True).na.drop()

    df.createOrReplaceTempView(bronze_sql_name)

    df.write.partitionBy("GENDER", "RACE", "ETHNIC", "MARSTAT", "EMPLOY").mode(
        "overwrite"
    ).parquet(bronze_output_path)

In [8]:
raw_path = os.path.join("..","data", "*.csv")
bronze_path = os.path.join("..","data", "medallion_layers_main", "bronze")
silver_path = os.path.join("..","data", "medallion_layers_main", "silver")
gold_path = os.path.join("..","data", "medallion_layers_main", "gold")
bronze_df = bronze_layer(
    raw_input_path=raw_path,
    bronze_output_path=bronze_path,
    bronze_sql_name="bronze",
)

24/05/25 18:09:20 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
ERROR:root:KeyboardInterrupt while sending command.                 (0 + 8) / 8]
Traceback (most recent call last):
  File "/Users/aymansulaiman/miniforge3/envs/crisis_text_line/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/Users/aymansulaiman/miniforge3/envs/crisis_text_line/lib/python3.10/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/Users/aymansulaiman/miniforge3/envs/crisis_text_line/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
df = spark.read.parquet

In [9]:
spark.stop()

24/05/25 18:11:00 ERROR FileFormatWriter: Aborting job a2e19474-1f04-4090-836d-3e7a5a02cd7f.
org.apache.spark.SparkException: Job 2 cancelled because SparkContext was shut down
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$cleanUpAfterSchedulerStop$1(DAGScheduler.scala:1253)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$cleanUpAfterSchedulerStop$1$adapted(DAGScheduler.scala:1251)
	at scala.collection.mutable.HashSet.foreach(HashSet.scala:79)
	at org.apache.spark.scheduler.DAGScheduler.cleanUpAfterSchedulerStop(DAGScheduler.scala:1251)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.onStop(DAGScheduler.scala:3087)
	at org.apache.spark.util.EventLoop.stop(EventLoop.scala:84)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$stop$3(DAGScheduler.scala:2973)
	at org.apache.spark.util.Utils$.tryLogNonFatalError(Utils.scala:1375)
	at org.apache.spark.scheduler.DAGScheduler.stop(DAGScheduler.scala:2973)
	at org.apache.spark.SparkContext.$anonfun$stop$12(SparkContex

24/05/25 18:11:01 ERROR Utils: Uncaught exception in thread Executor task launch worker for task 1.0 in stage 2.0 (TID 10)
java.lang.NullPointerException
	at org.apache.spark.scheduler.Task.$anonfun$run$3(Task.scala:146)
	at org.apache.spark.util.Utils$.tryLogNonFatalError(Utils.scala:1375)
	at org.apache.spark.scheduler.Task.run(Task.scala:144)
	at org.apache.spark.executor.Executor$TaskRunner.$anonfun$run$4(Executor.scala:620)
	at org.apache.spark.util.SparkErrorUtils.tryWithSafeFinally(SparkErrorUtils.scala:64)
	at org.apache.spark.util.SparkErrorUtils.tryWithSafeFinally$(SparkErrorUtils.scala:61)
	at org.apache.spark.util.Utils$.tryWithSafeFinally(Utils.scala:94)
	at org.apache.spark.executor.Executor$TaskRunner.run(Executor.scala:623)
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1149)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:624)
	at java.lang.Thread.run(Thread.java:748)
24/05/25 18:11:01 ERROR Executor: Except